## 한국수출입은행 - 원화 환율(기간 지정) 및 DB 저장
+ 작성: 임경호
+ 한국수출입은행 Open API https://www.koreaexim.go.kr/ir/HPHKIR019M01

In [13]:
# 2023년 1월 1일부터 2023년 6월 30일까지 데이터 일괄 저장
date_from = "20230601"    
date_to = "20230630"
url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON'
api_key = 'OczbdUJk31QuSPZNx3llq7VFOEmsMuew'
RESULT_SUCCESS = 1

In [14]:
import requests
import pandas as pd

date_range = pd.date_range(start=date_from, end=date_to, freq="D")

df_fx_all = pd.DataFrame()
for dt in date_range:
    date_str = dt.strftime("%Y%m%d")
    params = {
        'authkey': api_key,
        'searchdate': date_str,
        'data': 'AP01'             # 환율
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:     # URL GET '200 정상'
        print("URL GET Error")
        pass
    df_fx = pd.json_normalize(response.json())
    if not df_fx.empty:
        # print(df_fx['result'].unique())
        if RESULT_SUCCESS not in df_fx['result'].unique():         # 성공이 아니라면
            print(f'{date_str}: Result is not successful')
            pass
        df_fx.drop('result', axis=1, inplace=True)
        df_fx.insert(0, 'date', date_str)
        df_fx_all = pd.concat([df_fx_all, df_fx], ignore_index=True)

+ RESULT  Integer 조회결과
  - 1 : 성공, 2 : DATA코드 오류, 3 : 인증코드 오류, 4 : 일일제한횟수 마감
+ CUR_UNIT	String	통화코드
+ CUR_NM	String	국가/통화명
+ TTB String 전신환(송금) 받으실때
+ TTS String 전신환(송금) 보내실때
+ DEAL_BAS_R  String 매매 기준율
+ BKPR  String	장부가격	
+ YY_EFEE_R String	년환가료율	
+ TEN_DD_EFEE_R String	10일환가료율	
+ KFTC_DEAL_BAS_R String	서울외국환중개 매매기준율	
+ KFTC_BKPR String	서울외국환중개 장부가격

In [15]:
df_fx_all.shape

(483, 11)

In [16]:
# df_fx_all.groupby('date').agg(cur = ('cur_unit', 'count'))

In [17]:
# currency = 'USD'
# df_fx_all.query('cur_unit == @currency')

In [7]:
df_fx_all.to_excel('korea_eximbank_fx.xlsx')

#### DB에 저장

In [16]:
df_fx_all_save = df_fx_all[['date', 'cur_unit', 'ttb', 'tts', 'deal_bas_r', 'bkpr', 'kftc_bkpr', 'kftc_deal_bas_r']]

* 데이터 전처리

In [17]:
df_fx_all_save.dtypes

date               object
cur_unit           object
ttb                object
tts                object
deal_bas_r         object
bkpr               object
kftc_bkpr          object
kftc_deal_bas_r    object
dtype: object

In [18]:
# 숫자의 소수점(,) 없애기
df_fx_all_save.iloc[:,2:8] = df_fx_all_save.iloc[:,2:8].apply(lambda x: x.str.replace(',', ''), axis = 1)
# 통화코드에서 세 자리 초과하는 내용은 지우고 통화코드 생성: (예) JPY(100) -> JPY로 변경
df_fx_all_save.insert(2, 'cur_code', df_fx_all_save['cur_unit'].str[0:3])
df_fx_all_save = df_fx_all_save.drop('cur_unit', axis=1)

In [19]:
df_fx_all_save

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
0,20230601,AED,356.22,363.41,359.82,359,359,359.82
1,20230601,AUD,851.37,868.56,859.97,859,859,859.97
2,20230601,BHD,3470.23,3540.34,3505.29,3505,3505,3505.29
3,20230601,BND,968.13,987.68,977.91,977,977,977.91
4,20230601,CAD,964.31,983.8,974.06,974,974,974.06
...,...,...,...,...,...,...,...,...
478,20230630,SAR,346.52,353.53,350.03,350,350,350.03
479,20230630,SEK,119.61,122.02,120.82,120,120,120.82
480,20230630,SGD,958.28,977.63,967.96,967,967,967.96
481,20230630,THB,36.47,37.2,36.84,36,36,36.84


* DB 테이블에 저장

In [12]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_fx_all_save.itertuples():
    sql = "insert into fx_eximbank (date, cur_code, ttb, tts, deal_bas_r, bkpr, kftc_bkpr, kftc_deal_bas_r) \
                           values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]))
conn.commit()
conn.close()